In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score

/srv/home/trokhymovych/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dfs = [pd.read_csv(f"../data/2024-04_content_batch_{i}.csv") for i in tqdm(range(1,16))]
df = pd.concat(dfs).reset_index(drop=True)
print([len(d) for d in dfs])
print(len(df))

In [ ]:
# loading text features: 
df_text = pd.read_csv("../data/mlm_text_features.csv")

# loading holdout dataset: 
holdout_test = pd.read_csv("../data/holdout/test_holdout.csv", sep="\t")
revs_check = set(holdout_test.rev)

## Attempt to build the catboost model on-top: 


In [ ]:
df_cat_full = pd.read_csv("../data/2024-04_metadata.csv")

/tmp/ipykernel_1069401/1562895737.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cat_full = pd.read_csv("data/2024-04_metadata.csv")


In [ ]:
df["is_add"] = df.added.apply(lambda x: x != "{}")
df["is_remove"] = df.removed.apply(lambda x: x != "{}")
df["is_change"] = df.changed.apply(lambda x: x != "{}")
df["is_labels"] = df.labels.apply(lambda x: x != "{}")
df["is_descriptions"] = df.descriptions.apply(lambda x: x != "{}")

tmp_df = deepcopy(df)
np.random.seed(42)
unique_ids_all = list(df.page_title.unique())
ids_train_lm = list(np.random.choice(unique_ids_all, size=int(0.8*len(unique_ids_all)), replace=False))

df_cat = df_cat_full[df_cat_full.revision_id.isin(set(tmp_df.revision_id))]
df_cat["is_lm_train"] = df_cat.page_title.isin(ids_train_lm)

print(len(df_cat))
print(len(tmp_df))

6410636
6410636


/tmp/ipykernel_1069401/3216782044.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat["is_lm_train"] = df_cat.page_title.isin(ids_train_lm)


In [6]:
y_col = 'revision_is_identity_reverted'

cat_features = [
       'user_is_bot',
       'user_is_anonymous',
       'event_user_groups-bot',
       'event_user_groups-bureaucrat', 'event_user_groups-checkuser',
       'event_user_groups-confirmed', 'event_user_groups-flood',
       'event_user_groups-interface-admin', 'event_user_groups-ipblock-exempt',
       'event_user_groups-propertycreator', 'event_user_groups-rollbacker',
       'event_user_groups-suppress', 'event_user_groups-sysop',
       'event_user_groups-translationadmin', 'event_user_groups-wikidata-staff',
       # 'parent_same_user',  # <- Optional, as thould be implemented on revert risk lib
       # 'taggs'  # <- skipped
]

numeric_features = [
    "user_age",
    "page_seconds_since_previous_revision",
    "page_age",
]

text_features_mean = ["add_score_mean", "remove_score_mean", "change_score_mean"]
text_features_max = ["add_score_max", "remove_score_max", "change_score_max"]

In [ ]:
# Features preparation:

NUMERIC_NaN = -999
CATEGORICAL_NaN = "NaN"

## Text features: 
bert_scores_mean_dict = df_text.groupby(["revision_id", "action_type"]).scores.mean().to_dict()
bert_scores_max_dict = df_text.groupby(["revision_id", "action_type"]).scores.max().to_dict()

df_cat["add_score_mean"] = df_cat.revision_id.apply(lambda x: bert_scores_mean_dict.get((x, 'add: '), NUMERIC_NaN))
df_cat["remove_score_mean"] = df_cat.revision_id.apply(lambda x: bert_scores_mean_dict.get((x, 'remove: '), NUMERIC_NaN))
df_cat["change_score_mean"] = df_cat.revision_id.apply(lambda x: bert_scores_mean_dict.get((x, 'change: '), NUMERIC_NaN))

df_cat["add_score_max"] = df_cat.revision_id.apply(lambda x: bert_scores_max_dict.get((x, 'add: '), NUMERIC_NaN))
df_cat["remove_score_max"] = df_cat.revision_id.apply(lambda x: bert_scores_max_dict.get((x, 'remove: '), NUMERIC_NaN))
df_cat["change_score_max"] = df_cat.revision_id.apply(lambda x: bert_scores_max_dict.get((x, 'change: '), NUMERIC_NaN))

## Categorical Features Renaming: 
df_cat["user_is_bot"] = df_cat["is_bot"]
df_cat["user_is_anonymous"] = df_cat["event_user_is_anonymous"]


# Filling NaNs:
df_cat[cat_features] = df_cat[cat_features].astype(str).fillna(CATEGORICAL_NaN)
df_cat[numeric_features] = df_cat[numeric_features].fillna(NUMERIC_NaN)

In [8]:
## Train-Test split: 

ids_to_drop = df[(df.revision_is_identity_reverted == True) & ((df.self_revert) | (df.reverting_revision_is_reverted_revision))]["revision_id"].values
ids_train = df[
    (pd.to_datetime(df['event_timestamp']) < pd.to_datetime('2023-06-01')) & 
    (pd.to_datetime(df['event_timestamp']) > pd.to_datetime('2021-09-01'))
].revision_id
ids_holdout = df[
    (pd.to_datetime(df['event_timestamp']) > pd.to_datetime('2023-06-01')) & 
    (pd.to_datetime(df['event_timestamp']) < pd.to_datetime('2023-09-01'))
].revision_id

test = df_cat[~df_cat.revision_id.isin(ids_to_drop)]  # Filter self reverts
test = test[~test.revision_id.isin(revs_check)]  # filter experts scores
test = test[test.revision_id.isin(ids_holdout)]  # filter time

train = df_cat[~df_cat.revision_id.isin(ids_to_drop)]  # Filter self reverts
train = train[~train.revision_id.isin(revs_check)]  # filter experts scores
train = train[train.revision_id.isin(ids_train)]  # filter time
train = train[~train.is_lm_train]  # filter revisions used in train LM

val = df_cat[df_cat.revision_id.isin(revs_check)]

In [20]:
len(train)

838245

In [21]:
len(test)

645264

In [22]:
len(val)

1293

# Dummy model: 

In [ ]:
# Training the model: 
all_features_to_train = cat_features + numeric_features
print(f"Train features: {all_features_to_train}")

train_data = Pool(
    data=train[all_features_to_train],
    label=train[y_col].astype(int),
    cat_features=cat_features
)
test_data = Pool(
    data=test[all_features_to_train],
    label=test[y_col].astype(int),
    cat_features=cat_features
)

# balancing
classes = np.unique(train[y_col].astype(int))
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train[y_col].astype(int))
class_weights = dict(zip(classes, weights))

# Initialize CatBoostClassifier:
model_text = CatBoostClassifier(iterations=2500, metric_period=100, verbose=True, learning_rate=0.005,
                           class_weights=class_weights, thread_count=30, use_best_model=True,
                           custom_metric=['Logloss', 'AUC', 'Precision', 'Recall', 'F1', 'Accuracy'])
# Fit model
model_text.fit(train_data, eval_set=test_data, plot=False)
model_text.save_model('../models/dummy_full_new.cbn')

Train features: ['user_is_bot', 'user_is_anonymous', 'event_user_groups-bot', 'event_user_groups-bureaucrat', 'event_user_groups-checkuser', 'event_user_groups-confirmed', 'event_user_groups-flood', 'event_user_groups-interface-admin', 'event_user_groups-ipblock-exempt', 'event_user_groups-propertycreator', 'event_user_groups-rollbacker', 'event_user_groups-suppress', 'event_user_groups-sysop', 'event_user_groups-translationadmin', 'event_user_groups-wikidata-staff', 'user_age', 'page_seconds_since_previous_revision', 'page_age']
0:	learn: 0.6898161	test: 0.6901794	best: 0.6901794 (0)	total: 457ms	remaining: 19m 2s
100:	learn: 0.4880878	test: 0.5046926	best: 0.5046926 (100)	total: 27.1s	remaining: 10m 44s
200:	learn: 0.4284393	test: 0.4487530	best: 0.4487530 (200)	total: 53.4s	remaining: 10m 10s
300:	learn: 0.4083730	test: 0.4302717	best: 0.4302717 (300)	total: 1m 18s	remaining: 9m 31s
400:	learn: 0.3999661	test: 0.4229186	best: 0.4229186 (400)	total: 1m 42s	remaining: 8m 58s
500:	lear

In [ ]:
# Training the model: 
all_features_to_train = cat_features + numeric_features + text_features_mean + text_features_max
print(f"Train features: {all_features_to_train}")

train_data = Pool(
    data=train[all_features_to_train],
    label=train[y_col].astype(int),
    cat_features=cat_features
)
test_data = Pool(
    data=test[all_features_to_train],
    label=test[y_col].astype(int),
    cat_features=cat_features
)

# balancing
classes = np.unique(train[y_col].astype(int))
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train[y_col].astype(int))
class_weights = dict(zip(classes, weights))

# Initialize CatBoostClassifier:
model_text = CatBoostClassifier(iterations=2500, metric_period=100, verbose=True, learning_rate=0.005,
                           class_weights=class_weights, thread_count=30, use_best_model=True,
                           custom_metric=['Logloss', 'AUC', 'Precision', 'Recall', 'F1', 'Accuracy'])
# Fit model
model_text.fit(train_data, eval_set=test_data, plot=False)
model_text.save_model('../models/graph2text_full_new.cbn')

Train features: ['user_is_bot', 'user_is_anonymous', 'event_user_groups-bot', 'event_user_groups-bureaucrat', 'event_user_groups-checkuser', 'event_user_groups-confirmed', 'event_user_groups-flood', 'event_user_groups-interface-admin', 'event_user_groups-ipblock-exempt', 'event_user_groups-propertycreator', 'event_user_groups-rollbacker', 'event_user_groups-suppress', 'event_user_groups-sysop', 'event_user_groups-translationadmin', 'event_user_groups-wikidata-staff', 'user_age', 'page_seconds_since_previous_revision', 'page_age', 'add_score_mean', 'remove_score_mean', 'change_score_mean', 'add_score_max', 'remove_score_max', 'change_score_max']
0:	learn: 0.6888418	test: 0.6892994	best: 0.6892994 (0)	total: 340ms	remaining: 14m 9s
100:	learn: 0.4455518	test: 0.4716115	best: 0.4716115 (100)	total: 26.5s	remaining: 10m 29s
200:	learn: 0.3774593	test: 0.4054228	best: 0.4054228 (200)	total: 52s	remaining: 9m 54s
300:	learn: 0.3514354	test: 0.3794544	best: 0.3794544 (300)	total: 1m 17s	remai

In [ ]:
# Training the model: 
numeric_features_no_user = [
    "page_seconds_since_previous_revision",
    "page_age",
]

all_features_to_train = text_features_mean + text_features_max + numeric_features_no_user
print(f"Train features: {all_features_to_train}")

train_data = Pool(
    data=train[all_features_to_train],
    label=train[y_col].astype(int),
    cat_features=[]
)
test_data = Pool(
    data=test[all_features_to_train],
    label=test[y_col].astype(int),
    cat_features=[]
)

# balancing
classes = np.unique(train[y_col].astype(int))
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train[y_col].astype(int))
class_weights = dict(zip(classes, weights))

# Initialize CatBoostClassifier:
model_text_only = CatBoostClassifier(iterations=2500, metric_period=100, verbose=True, learning_rate=0.005,
                           class_weights=class_weights, thread_count=30, use_best_model=True,
                           custom_metric=['Logloss', 'AUC', 'Precision', 'Recall', 'F1', 'Accuracy'])
# Fit model
model_text_only.fit(train_data, eval_set=test_data, plot=False)
model_text_only.save_model('../models/graph2text_only_full_new.cbn')

Train features: ['add_score_mean', 'remove_score_mean', 'change_score_mean', 'add_score_max', 'remove_score_max', 'change_score_max', 'page_seconds_since_previous_revision', 'page_age']
0:	learn: 0.6898269	test: 0.6901261	best: 0.6901261 (0)	total: 57ms	remaining: 2m 22s
100:	learn: 0.5026932	test: 0.5227993	best: 0.5227993 (100)	total: 2.45s	remaining: 58.2s
200:	learn: 0.4447764	test: 0.4728503	best: 0.4728503 (200)	total: 4.84s	remaining: 55.4s
300:	learn: 0.4214773	test: 0.4537155	best: 0.4537155 (300)	total: 7.26s	remaining: 53s
400:	learn: 0.4104149	test: 0.4448052	best: 0.4448052 (400)	total: 9.69s	remaining: 50.7s
500:	learn: 0.4043433	test: 0.4399487	best: 0.4399487 (500)	total: 12s	remaining: 47.9s
600:	learn: 0.4004679	test: 0.4370720	best: 0.4370720 (600)	total: 14.3s	remaining: 45.3s
700:	learn: 0.3976705	test: 0.4351118	best: 0.4351118 (700)	total: 16.7s	remaining: 42.8s
800:	learn: 0.3955601	test: 0.4338069	best: 0.4338069 (800)	total: 19s	remaining: 40.3s
900:	learn: 0.